# 🧪 Hands-on Lab: Exploring Snowflake Data Science and ML Capabilities

Welcome to this **Hands-on Lab**, where you'll get practical experience with some of Snowflake’s most powerful features for Data Science and Machine Learning development:

### 🔹 Snowflake Notebooks  
Learn how to interactively explore, analyze, and visualize data using Snowflake Notebooks — an integrated development environment (IDE) right within the Snowflake UI. With support for Python and SQL side by side, Notebooks make it easier than ever to build, test, and document your data workflows.

### 🔹 Snowflake Cortex ML Functions  
Unlock the power of Snowflake Cortex, a fully managed machine learning service. You'll discover how to apply pre-built Cortex ML functions for tasks like classification, translation, task completion, and more — all without leaving the Snowflake environment.

### 🔹 Snowflake Model Registry  
Take your ML projects to the next level with the **Model Registry**, which lets you manage and deploy custom models built inside or outside of Snowflake. You’ll learn how to register, version, and serve models, enabling seamless collaboration and production-ready deployments.

---

### ✅ By the end of this lab, you will:
- Build and run interactive notebooks using Python and SQL  
- Use Cortex ML functions to derive insights from raw data  
- Register and deploy a custom machine learning model  
---

### 🗓️ Lab Agenda

This hands-on lab is designed to guide you through the full cycle of building and operationalizing machine learning workflows directly in Snowflake. Below is the step-by-step agenda we'll follow:
- 1️⃣ Data Sources Description  
- 2️⃣ Data Visualization and Manipulation  
- 3️⃣ Snowflake ML Capabilities  
- 4️⃣ Snowflake Model Registry

Let’s dive in and start building with AI in the Data Cloud! ☁️💡

## ⚙️ Lab Setup Instructions
Before we dive into the hands-on lab, please follow the steps below to get your environment ready. These steps ensure you have access to the right tools, permissions, and datasets.

1. Please make sure you are using the role ***TRAINING_03_2025_ROLE***
2. Please validate that your notebook is prefixed with your first and last name.
3. Click on the ***Packages*** button at the upper corner of the screen and make sure you have selected the following packages
    - snowflake-ml-python==1.7.4
    - python==3.9
    - streamlit==1.39.1
    - plotly==5.24.1
4. Click on ***Notebook Settings*** in the ellipsis on the top right corner of the window and ensure you are using warehouse ***TRAINING_03_2025_WH_1*** or ***TRAINING_03_2025_WH_2*** as instructed by the lab professor.
5. Finally, please change the variable ***schema_name*** bellow with the name of the schema prefix with your first and last name.


In [ ]:
# Import python packages
from snowflake.ml.registry import Registry
from snowflake.ml.modeling.xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
from snowflake.snowpark.context import get_active_session
import streamlit as st
import plotly.express as px
import pandas as pd
import warnings

# Disable warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Get a Snowpark Session
session = get_active_session()


### ⚠️ Important

Be sure to **replace the schema name** in the sample code with **your own schema name** to avoid errors and ensure you're working within the correct context.

In [ ]:
schema_name = 'add_your_schema_name_here'

In [ ]:
-- CHANGE THIS TO USE THE ATTENDANCE SCHEMA.
USE DATABASE HOL_LAB_DATASCIENCE;
USE SCHEMA {{schema_name}};

# 1️⃣ Data Sources Description 
- Overview of the dataset(s) used in this lab  
- Schema and data types  
- Use cases and business context for the analysis

For this lab, we will use two demo tables:
1. HOL_LAB_DATASCIENCE.LAB.CLUSTER_QUALIFICATION
2. HOL_LAB_DATASCIENCE.LAB.pilot_1_e_commerce_comments

In [ ]:
SELECT
	CLUSTER,
	CM_OWN_WAY,
	CM_SECURE_SURROUNDINGS,
	CM_GOOD_TIME,
	CM_FOLLOW_RULES,
	CM_DIFFERENT_PEOPLE,
	CM_SATISFACTION,
	CM_SUCCESSFUL,
	CM_DEVOTE_MYSELF,
	CM_TAKE_RISKS,
	CM_BE_IN_CHARGE
FROM HOL_LAB_DATASCIENCE.LAB.CLUSTER_QUALIFICATION
WHERE cluster is not null;

In [ ]:
SELECT 
    key_voysen,
    name_voysen,
    brand_voysen,
    name_source,
    date_datetime,
    review_translated_text,
    topics 
FROM TRAINING_03_2025_DB.DEMO.PILOT_1_E_COMMERCE_COMMENTS;

# 2️⃣ Data Visualization and Manipulation   
- Exploring and querying the data with SQL and Python  
- Basic visualizations using built-in charting tools  
- Data cleaning and transformation using Python, SQL and Snowflake Cortex LLM Functions

### DATASET PILOT_1_PRODUCT_TEST

In [ ]:
cluster_qualification_view = f'HOL_LAB_DATASCIENCE.{schema_name}.CLUSTER_QUALIFICATION'

This SQL statement creates or replaces a **view** named `cluster_qualification_view`. The purpose of this view is to summarize and analyze **participant feedback** or **qualification scores** by **cluster**, providing average values for a series of CM (Customer Metrics or Qualification Metrics) fields.

In [ ]:
CREATE OR REPLACE VIEW {{cluster_qualification_view}} AS (
    SELECT
    	CLUSTER,
    	AVG(CM_OWN_WAY) AS CM_OWN_WAY_AVG,
        AVG(CM_SECURE_SURROUNDINGS) AS CM_SECURE_SURROUNDINGS_AVG,
        AVG(CM_GOOD_TIME) AS CM_GOOD_TIME_AVG,
        AVG(CM_FOLLOW_RULES) AS CM_FOLLOW_RULES_AVG,
        AVG(CM_DIFFERENT_PEOPLE) AS CM_DIFFERENT_PEOPLE_AVG,
        AVG(CM_SATISFACTION) AS CM_SATISFACTION_AVG,
        AVG(CM_SUCCESSFUL) AS CM_SUCCESSFUL_AVG,
        AVG(CM_DEVOTE_MYSELF) AS CM_DEVOTE_MYSELF_AVG,
        AVG(CM_TAKE_RISKS) AS CM_TAKE_RISKS_AVG,
        AVG(CM_BE_IN_CHARGE) AS CM_BE_IN_CHARGE 
    FROM HOL_LAB_DATASCIENCE.LAB.CLUSTER_QUALIFICATION
    WHERE CLUSTER is not null
    GROUP BY CLUSTER
)

In [ ]:
SELECT * FROM {{cluster_qualification_view}}

When working with multiple numerical vectors, it's often useful to compare how similar or different they are from one another. This can help in tasks like clustering, recommendation systems, or pattern recognition. Snowflake Notebooks facilitates this process by providing an easy way to interact between SQL and Python cells.


In [ ]:
df = cluster_qualification.to_pandas()
good_friend = df[df["CLUSTER"] == "Good Friend"].iloc[0,1:].values
control_freak = df[df["CLUSTER"] == "Control Freak"].iloc[0,1:].values
cosine_similarity([good_friend], [control_freak])[0][0]

### DATASET PILOT_1_E_COMMERCE_COMMENTS

Snowflake has the ability to work directly with semi-structured data in SQL. This allows us to unpack JSON data without having to build out elaborate structures and more quickly extract information.

In [ ]:
top_mention_view = f'TRAINING_03_2025_DB.{schema_name}.TOP_MENTION'

In [ ]:
SELECT
    TOP 100 *
FROM
    TRAINING_03_2025_DB.DEMO.pilot_1_e_commerce_comments;
CREATE OR REPLACE VIEW {{top_mention_view}} AS (
        WITH parsed AS (
            SELECT
                PARSE_JSON(topics) AS p_topics,
                *
            FROM
                TRAINING_03_2025_DB.DEMO.pilot_1_e_commerce_comments
        ),
        flattened AS (
            SELECT
                p.p_topics:positive AS positive,
                p.p_topics:"mentionned_topics" AS mentioned_list,
                topic.value AS topic,
                p.*
            FROM
                parsed p,
                LATERAL FLATTEN (input => p_topics:mentionned_topics) m,
                LATERAL FLATTEN(strtok_to_array(m.value, ', ')) topic
        )
        SELECT
            name_voysen,
            topic,
            count(*) AS "Number of Mentions"
        FROM
            flattened
        GROUP BY
            name_voysen,
            topic
    );

By building out the intermediate data models, we can build more interesting models on top of them to help gain insights.

Note that the cell names will be used as references in subsequent cells.

In [ ]:
select 
    
    Name_Voysen as Product, 
    Topic, 
    "Number of Mentions" as Mentions
from {{top_mention_view}}
QUALIFY RANK() OVER (PARTITION BY Name_Voysen ORDER BY "Number of Mentions" DESC) <=3
order by Name_Voysen, "Number of Mentions" DESC;


Let's visualize our data by using Streamlit to display charts of the data we queried earlier using the cell names of the notebook as references.

In [ ]:
# this converts the results from the execution of the cell specified into data we can feed into our chart
my_df = TOPICS_MENTIONED_BY_PRODUCT.to_pandas()

# Chart the data
st.subheader("Top 3 Topics Mentioned by Product")
st.bar_chart(my_df, x="PRODUCT", y="MENTIONS", color='TOPIC', stack=False)



We can use more advanced Python libraries, such as Plotly, to create richer and more interactive visualizations.

In [ ]:
# Convert Snowpark DataFrame to pandas DataFrame
my_df = TOPICS_MENTIONED_BY_PRODUCT.to_pandas()

fig = px.bar(
    my_df,
    x="PRODUCT",
    y="MENTIONS",
    color="TOPIC",
    barmode="group",  # Use "stack" for stacked bars
    title="Top 3 Topics Mentioned by Product",
    labels={"PRODUCT": "Product", "MENTIONS": "Number of Mentions", "TOPIC": "Topic"},
)

st.plotly_chart(fig, use_container_width=True)

# Data manipulation with Snowflake Cortex LLM Functions

In this example, we demonstrate how to use **Snowflake Cortex LLM functions** to classify and clean free-text entries in a dataset, specifically leveraging the `CLASSIFY_TEXT` function to standardize product names.

---

## 🧠 What is `CLASSIFY_TEXT`?

`SNOWFLAKE.CORTEX.CLASSIFY_TEXT` is a Large Language Model (LLM) function provided by Snowflake Cortex. It allows you to match input text to the closest label in a list of predefined categories using AI, making it ideal for:
- Text classification
- Entity resolution
- Standardization of messy input data

In [ ]:
WITH base_query AS (
    SELECT 
        PARSE_JSON(SNOWFLAKE.CORTEX.CLASSIFY_TEXT(
            replace(name_source, '?', ''), 
            ['Chanel', 'Chanel N5', 'Dior', 'Chanel Chance', 'Chanel 520', 
             'Dior  Fahrenheit', 'Dior Ambre Nuit', ' Chanel Sensuelle', 
             'Chanel Allure', 'Chanel Allure Sport', 'Chanel Allure Sensuelle', 
             'Chanel Les 1957', 'ER', '72']
        )):label::string AS name_source,
        replace(name_source, '?', '') AS original_name_source
    FROM TRAINING_03_2025_DB.DEMO.PILOT_1_E_COMMERCE_COMMENTS
    LIMIT 1000
)
SELECT name_source, original_name_source 
FROM base_query;

In this other example, we use **Snowflake Cortex’s `COMPLETE` LLM function** to enhance the grammar of customer reviews while preserving the original tone and meaning. This is especially useful for cleaning up user-generated content before using it in dashboards, machine learning models, or customer-facing applications.

---

## 🤖 What is `SNOWFLAKE.CORTEX.COMPLETE`?

The `COMPLETE` function allows you to interact with state-of-the-art Large Language Models (LLMs), like `mistral-large`, by passing **custom prompts**. This gives you flexibility to:
- Rewrite or transform text
- Generate structured output
- Summarize, translate, or correct grammar
- Answer domain-specific questions

In [ ]:
SELECT 
    review_translated_text AS review_translated_text_original,
    SNOWFLAKE.CORTEX.COMPLETE(
        'mistral-large',
        CONCAT(
            'Correct the grammar of the following review, but keep the corrected version as similar as possible to the original review version. ',
            'Do not add any explanations or labels. Only output the corrected version of the text: ',
            review_translated_text,
            ' Output:'
        )
    ) AS review_translated_text_improved
FROM TRAINING_03_2025_DB.DEMO.PILOT_1_E_COMMERCE_COMMENTS
LIMIT 10

We can combine multiple LLM function calls to accomplish more complex tasks. For example, we can translate the improved review text to French by just calling the Snowflake Cortex LLM function "translate".

## 🌐 What is `SNOWFLAKE.CORTEX.TRANSLATE`?

`SNOWFLAKE.CORTEX.TRANSLATE` is a built-in function that allows you to translate text between languages using AI models — directly within Snowflake.


In [ ]:
WITH base_query AS (
    SELECT 
        review_translated_text AS review_translated_text_original,
        SNOWFLAKE.CORTEX.COMPLETE(
            'mistral-large',
            CONCAT(
                'Correct the grammar of the following review, but keep the corrected version as similar as possible to the original review version. ',
                'Do not add any explanations or labels. Only output the corrected version of the text: ',
                review_translated_text,
                ' Output:'
            )
        ) AS review_translated_text_improved
    FROM TRAINING_03_2025_DB.DEMO.PILOT_1_E_COMMERCE_COMMENTS
    LIMIT 10
)
SELECT 
    review_translated_text_original,
    review_translated_text_improved,
    SNOWFLAKE.CORTEX.TRANSLATE(
        review_translated_text_improved,
        'en',
        'fr'
    ) AS review_translated_text_french
FROM base_query

# 3️⃣ Snowflake ML Capabilities  
- Overview of Snowflake Cortex ML functions  
- Build a sample ML model

First, let's define the data we will use for our machine learning use case. In this instance, we will utilize customer qualification scores to predict profile clusters. Before we proceed, let's use Snowflake to establish our model data table.

In [ ]:
model_data_table = f'TRAINING_03_2025_DB.{schema_name}.MODEL_DATA'

In [ ]:
CREATE OR REPLACE TABLE {{model_data_table}} AS (
    SELECT
        ROW_NUMBER() OVER (ORDER BY RANDOM(42)) AS id,
        CASE
            WHEN CLUSTER = 'Good Friend' THEN 0
            WHEN CLUSTER = 'Seeker' THEN 1
            WHEN CLUSTER = 'Super Achiever' THEN 2
            WHEN CLUSTER = 'Control Freak' THEN 3
            WHEN CLUSTER = 'Traditionalist' THEN 4
        END CLUSTER,
    	CM_OWN_WAY,
    	CM_SECURE_SURROUNDINGS,
    	CM_GOOD_TIME,
    	CM_FOLLOW_RULES,
    	CM_DIFFERENT_PEOPLE,
    	CM_SATISFACTION,
    	CM_SUCCESSFUL,
    	CM_DEVOTE_MYSELF,
    	CM_TAKE_RISKS,
    	CM_BE_IN_CHARGE
    FROM TRAINING_03_2025_DB.DEMO.PILOT_1_PRODUCT_TEST
    WHERE cluster IS NOT null 
    ORDER BY RANDOM(42)
)

Next, let's split our data into training and test sets so we can build the model.

In [ ]:
train_table = f'TRAINING_03_2025_DB.{schema_name}.TRAIN_DATA'
test_table = f'TRAINING_03_2025_DB.{schema_name}.TEST_DATA'

In [ ]:
CREATE OR REPLACE TABLE {{train_table}} AS
SELECT *
FROM {{model_data_table}}
SAMPLE (80);

CREATE OR REPLACE TABLE {{test_table}} AS
SELECT *
FROM {{model_data_table}}
WHERE id NOT IN (
    SELECT id FROM {{train_table}}
);

We can use the data in Snowflake to train our own model.

## 📚 Snowflake XGBoost Classifier

The model used is [`snowflake.ml.modeling.xgboost.XGBClassifier`](https://docs.snowflake.com/en/developer-guide/snowpark-ml/reference/1.0.9/api/modeling/snowflake.ml.modeling.xgboost.XGBClassifier), a Snowflake-provided wrapper around the popular XGBoost algorithm for classification tasks. It integrates seamlessly with Snowpark DataFrames and supports in-database machine learning workflows.

### 🔑 Key Features:
- Accepts both **Snowpark DataFrames** and **Pandas DataFrames**.
- Simplifies specifying **input**, **label**, and **output** columns.
- Supports common XGBoost hyperparameters (`n_estimators`, `max_depth`, etc.).
- Can be integrated into Snowflake's end-to-end ML lifecycle.

In [ ]:
train_df = session.table(train_table).to_pandas().head(1000) # We are using only 1000 samples for testing. Feel free to increase this value.
test_df = session.table(test_table).to_pandas()

model = XGBClassifier(
    input_cols=['CM_OWN_WAY','CM_SECURE_SURROUNDINGS','CM_GOOD_TIME','CM_FOLLOW_RULES','CM_DIFFERENT_PEOPLE','CM_SATISFACTION','CM_SUCCESSFUL','CM_DEVOTE_MYSELF','CM_TAKE_RISKS','CM_BE_IN_CHARGE'],
    label_cols=['CLUSTER'],
    output_cols=["PREDICTED_CLUSTER"],
    n_estimators=200,
)

# Train
model.fit(train_df)

# Predict
result = model.predict(test_df)
result[['CLUSTER', 'PREDICTED_CLUSTER']].head()

Finaly, let's verify the accuracy of the model.

In [ ]:
acc = accuracy_score(result["CLUSTER"], result["PREDICTED_CLUSTER"])
acc

# 4️⃣ Snowflake Model Registry  
- Introduction to the Model Registry  
- Registering and versioning a custom model  
- Deploying a model for inference  
- Tracking model metadata and performance metrics  
- Invoking the model through SQL and Python  

## 🎯 What is the Model Registry?
The **Snowflake Model Registry** is a centralized system to **store, manage, and deploy machine learning models** within Snowflake. It enables secure, versioned, and collaborative model management as part of a complete in-database ML lifecycle using **Snowpark ML**.

The Model Registry in Snowflake is designed to:
- **Register trained models** for future use.
- **Version models** for better experiment tracking.
- **Store metadata** like model name, version, input/output schema, and tags.
- **Share models** across teams and projects.
- **Deploy models** directly within Snowflake for scoring on Snowpark DataFrames.

It allows data scientists and ML engineers to **reuse and deploy models consistently**, all while keeping data and model artifacts inside the Snowflake Data Cloud.


In [ ]:
reg = Registry(session=session, database_name="TRAINING_03_2025_DB", schema_name=schema_name)
model_name = 'CLUSTER_CLASSIFICATION'

This code registers the trained model into the **Snowflake Model Registry**, enabling versioning, reuse, and in-database deployment.

In [ ]:
# Let's first log the very first model we trained
model_ver = reg.log_model(
    model_name=model_name,
    version_name='V7',
    model=model,
    sample_input_data=train_df.head(100), # to provide the feature schema
    options={"enable_explainability": True, 'relax_version': True}
)

We can include evaluation metrics and descriptive comments when registering a model to improve metadata tracking and documentation. Additionally, all registered models and their versions can be visualized directly in the Snowflake Model Registry, making it easier to monitor and manage the ML lifecycle.

In [ ]:
# Add evaluation metric
model_ver.set_metric(metric_name="accuracy", value=acc)

# Add a description
model_ver.comment = "This is the first iteration of our Cluster Multi-Class model. It is used for demo purposes."

# Let's confirm they were added
reg.get_model(model_name).show_versions()


Finally, we can load the registered model by specifying its version, using either Python or SQL, depending on the deployment or inference workflow.

In [ ]:
model_ver = reg.get_model(model_name).version('v1')
result_sdf2 = model_ver.run(test_df, function_name="predict")
result_sdf2

In [ ]:
model_function = f'TRAINING_03_2025_DB.{schema_name}.CLUSTER_CLASSIFICATION!PREDICT'

In [ ]:
SELECT 
    cluster,
    {{model_function}}(
        CM_OWN_WAY,
        CM_SECURE_SURROUNDINGS,
        CM_GOOD_TIME,
        CM_FOLLOW_RULES,
        CM_DIFFERENT_PEOPLE,
        CM_SATISFACTION,
        CM_SUCCESSFUL,
        CM_DEVOTE_MYSELF,
        CM_TAKE_RISKS,
        CM_BE_IN_CHARGE
    ):PREDICTED_CLUSTER::string AS predicted_cluster
FROM TRAINING_03_2025_DB.DEMO.MODEL_DATA;